In [5]:
test = pd.read_pickle('housing_rental_ratio.pkl')
test = test.rename(columns={'RegionName':'zip'})
test['zip'] = test['zip'].astype(str)
test.head()

,zip,r_agg_2011,r_agg_2015_adj,r_pct_delta,h_agg_2011,h_agg_2015_adj,h_pct_delta,ratio_2011,ratio_2015,ratio_pct_delta
0,11368,1.868000,2.188278,1.171455,373.833333,431.274105,1.153653,200.124911,197.083754,0.984804
1,79936,0.744167,0.694379,0.933096,80.333333,75.060164,0.934359,107.950728,108.096860,1.001354
2,60629,1.197000,1.180207,0.985971,107.166667,113.430961,1.058454,89.529379,96.111037,1.073514
3,90650,1.524333,1.640026,1.075898,228.750000,292.549418,1.278905,150.065602,178.380911,1.188686
4,11226,1.014000,1.277783,1.260141,309.166667,416.876846,1.348389,304.898093,326.250103,1.070030


In [47]:
def countyMedian(pkl, col) :

    import numpy as np
    import json
    import pandas as pd
    test = pd.read_pickle(pkl)
    test['zip'] = test['zip'].astype(str)

    #processing zip codes
    lst=[]
    for i in test["zip"]:
        if len(str(i))!=5:
            lst.append("0"+str(i))
        else:
            lst.append(str(i))

    newcountyCodes=pd.read_csv('addZipCodes/zip_to_county.csv', dtype={'ZCTA5': object, "GEOID" : object})
    newcountyCodes = newcountyCodes.rename(columns={'ZCTA5': 'zip'})
    newcountyCodes['zip'] = newcountyCodes['zip'].astype(str)

    db = pd.merge(test, newcountyCodes, on='zip')

    def agg_metric(metric):
        gb = db[[metric, "GEOID"]].groupby("GEOID")
        medians = gb.agg([np.median])
        medians.head()

        #processing into json format
        lst = []
        for i in medians.iterrows():
            dct = {}
            dct["id"] = i[0]
            if np.isnan(i[1][0]):
                dct["value"] = None
            else:
                dct["value"] = i[1][0]
            lst.append(dct)

        for i in lst:
            if i["id"][0] == "0":
                i["id"] = i["id"][1:]

        import json
        fd=open("data/"+metric+".json","w")
        json.dump(lst, fd)
        fd.close()

        return medians, lst

    medians, lst = agg_metric(col)

    med_list = []
    for i in medians.iterrows():
        if np.isnan(i[1][0]):
            med_list.append(np.nan)
        else:
            med_list.append(i[1][0])

    s=pd.Series(med_list)

    #get 2011 house price data
    lst = []
    for i in medians.iterrows():
        dct = {}
        dct["id"] = i[0]
        if np.isnan(i[1][0]):
            dct["value"] = None
        else:
            dct["value"] = i[1][0]
        lst.append(dct)

    import json
    fd=open("data/houses_2011_3.json","w")
    json.dump(lst, fd)
    fd.close()

    countyCodes=pd.read_csv('addZipCodes/US_FIPS_Codes.csv')
    i = countyCodes[["FIPS State", "FIPS County"]]
    FIPs = []
    for row in i.iterrows():
        FIPs.append(('%02d' % row[1][0]) + ('%03d' % row[1][1]))
    countyCodes["FIP_Code"] = FIPs

    county_srted = db.sort('STATE')
    county_srted.head()

    new_db = pd.merge(db, newcountyCodes, on=['zip','GEOID','STATE','COUNTY'])
    gb = new_db.groupby("GEOID")
    new = gb.agg([np.median])

    return new[col].dropna()

In [49]:
countyMedian('housing_rental_ratio2.pkl', 'ratio_pct_delta')

,median
GEOID,
01001,1.005043
01003,0.993956
01007,0.859055
01009,0.955792
01013,1.133192
01019,1.220653
01021,0.868474
01031,0.784336
01037,0.917599
